## Notebook to create Unity Catalog functions, Bedrock Agents and Action groups.

In [1]:
import logging
import sys

from unitycatalog.ai.bedrock.envs.bedrock_env_vars import BedrockEnvVars
from unitycatalog.ai.bedrock.utils import pretty_print

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

bedrock_env = BedrockEnvVars()

In [2]:
bedrock_env.aws_profile = "default"
bedrock_env.aws_region = "us-east-1"
bedrock_env.bedrock_model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
bedrock_env.bedrock_rpm_limit = 1
bedrock_env.require_bedrock_confirmation = "DISABLED"

## Define Agentic Functions

In [6]:
def get_weather_in_celsius(location_id: str, fetch_date: str) -> str:
    """
    Fetches weather data (in Celsius) for a given location and date.

    Args:
        location_id (str): A unique identifier for the location.
        fetch_date (str): The date to fetch the weather for (format: YYYY-MM-DD).

    Raises:
        Exception: For unexpected errors during execution.

    Returns:
        str: Temperature in Celsius.
    """
    try:
        return str(23)
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}") from e

In [7]:
def get_weather_in_fahrenheit(location_id: str, fetch_date: str) -> str:
    """
    Fetches weather data (in fahrenheit) for a given location and date.

    Args:
        location_id (str): A unique identifier for the location.
        fetch_date (str): The date to fetch the weather for (format: YYYY-MM-DD).

    Raises:
        Exception: For unexpected errors during execution.

    Returns:
        str: Temperature in fahrenheit.
    """
    try:
        return str(72)
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}") from e

### Create Bedrock Agents and Action groups.

In [ ]:
import json
import logging
import sys
import time
import uuid

import boto3

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)


iam_client = boto3.client("iam")
sts_client = boto3.client("sts")
session = boto3.session.Session()

unique_id = str(uuid.uuid4())[:8]

In [10]:
bedrock_env.aws_region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# Create bedrock agent IAM policy
agent_bedrock_allow_policy_name = f"ucai-iam-policy-name-{unique_id}"

# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{bedrock_env.aws_region}::foundation-model/{bedrock_env.bedrock_model_id}"
            ],
        },
        {
            "Sid": "AmazonBedrockAgentStreamBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModelWithResponseStream",
            "Resource": [
                f"arn:aws:bedrock:{bedrock_env.aws_region}::foundation-model/{bedrock_env.bedrock_model_id}"
            ],
        },
    ],
}


bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name, PolicyDocument=bedrock_policy_json
)

logger.info("Bedrock Agent Policy:\n%s", pretty_print(agent_bedrock_policy))

In [ ]:
# Create IAM Role for the agent and attach IAM policies
agent_role_name = f"ucai-iam-role-name-{unique_id}"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name, AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name, PolicyArn=agent_bedrock_policy["Policy"]["Arn"]
)

In [13]:
# Create bedrock agent

bedrock_env.bedrock_agent_name = f"ucai-bedrock-agent-{unique_id}"
agent_description = "ucai-bedrock-agent-description"
agent_instruction = "You are a weather agent to fetch the current weather in celsius and fahrenheit for a given location"

bedrock_agent_client = boto3.client("bedrock-agent")

create_agent_response = bedrock_agent_client.create_agent(
    agentName=bedrock_env.bedrock_agent_name,
    agentResourceRoleArn=agent_role["Role"]["Arn"],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=bedrock_env.bedrock_model_id,
    instruction=agent_instruction,
)
# logger.info(create_agent_response)

In [15]:
bedrock_env.bedrock_agent_id = create_agent_response["agent"]["agentId"]

### Bind Agentic Function to Bedrock Action Group

In [16]:
def get_agent_functions(require_confirmation="DISABLED"):
    """
    Get agent functions with the specified confirmation requirement.

    Args:
        require_confirmation: "ENABLED" or "DISABLED" for function confirmation

    Returns:
        List of agent function definitions
    """
    return [
        {
            "name": "get_weather_in_celsius",
            "description": "Fetch the current weather in celsius for a given location and date",
            "parameters": {
                "location_id": {
                    "description": "The unique identifier of the location to retrieve the temperature for",
                    "required": True,
                    "type": "string",
                },
                "fetch_date": {
                    "description": "The specific date for which the temperature needs to be retrieved",
                    "required": True,
                    "type": "string",
                },
            },
            "requireConfirmation": require_confirmation,
        },
        {
            "name": "get_weather_in_fahrenheit",
            "description": "Fetch the current weather in fahrenheit for a given location and date",
            "parameters": {
                "location_id": {
                    "description": "The unique identifier of the location to retrieve the temperature for",
                    "required": True,
                    "type": "string",
                },
                "fetch_date": {
                    "description": "The specific date for which the temperature needs to be retrieved",
                    "required": True,
                    "type": "string",
                },
            },
            "requireConfirmation": require_confirmation,
        },
    ]

In [ ]:
agent_functions = get_agent_functions(bedrock_env.require_bedrock_confirmation)
logger.info(f"agent_functions:{agent_functions}")

In [20]:
# Prepare agent group using function schema
agent_action_group_name = f"ucai-bda-action-group-name-{unique_id}"
agent_action_group_description = "Actions to fetch the weather of a given location for a given date"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=bedrock_env.bedrock_agent_id,
    agentVersion="DRAFT",
    actionGroupExecutor={"customControl": "RETURN_CONTROL"},
    actionGroupName=agent_action_group_name,
    functionSchema={"functions": agent_functions},
    description=agent_action_group_description,
)

response = bedrock_agent_client.prepare_agent(agentId=bedrock_env.bedrock_agent_id)

In [21]:
# Create bedrock agent alias
agent_alias_name = f"ucai-bedrock-agent-alias-{unique_id}"
agent_alias_description = "Alias for the weather agent"

create_alias_response = bedrock_agent_client.create_agent_alias(
    agentId=bedrock_env.bedrock_agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)

bedrock_env.bedrock_agent_alias_id = create_alias_response['agentAlias']['agentAliasId']

### UC OSS Bedrock Agent Demo notebook 
### Shorter version to test existing agents and functions/tools.

In [22]:
CATALOG = "AICatalog"
SCHEMA = "AISchema"

In [23]:
def setup_uc_client():
    """Set up Unity Catalog client"""
    config = Configuration()
    config.host = "http://0.0.0.0:8080/api/2.1/unity-catalog"
    api_client = ApiClient(configuration=config)
    return UnitycatalogFunctionClient(api_client=api_client)

In [24]:
client = setup_uc_client()

In [ ]:
client.uc.create_catalog(name=CATALOG, comment="Catalog for AI functions")
client.uc.create_schema(catalog_name=CATALOG, name=SCHEMA, comment="Schema for AI functions")
client.create_python_function(
    func=get_weather_in_celsius, catalog=CATALOG, schema=SCHEMA, replace=True
)
client.create_python_function(
    func=get_weather_in_fahrenheit, catalog=CATALOG, schema=SCHEMA, replace=True
)
client.list_functions(catalog=CATALOG, schema=SCHEMA)

In [ ]:
function_name = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"
uc_func_toolkit = UCFunctionToolkit(function_names=[function_name], client=client)

logger.info(f"UC Function Toolkit:{uc_func_toolkit}")

In [ ]:
# Create a session
session_respone = uc_func_toolkit.create_session(
    agent_id=bedrock_env.bedrock_agent_id,
    agent_alias_id=bedrock_env.bedrock_agent_alias_id,
    catalog_name=CATALOG,
    schema_name=SCHEMA,
)
# Generate unique session ID
session_id = str(uuid.uuid1())

In [ ]:
final_response = session_respone.invoke_agent(
    input_text="What is the weather in Centigrade and Fahrenheit for location 12345 and date of 2025-02-26?",
    enable_trace=True,
    session_id=session_id,
    uc_client=uc_func_toolkit.client,
    model_id=bedrock_env.bedrock_model_id,
    rpm_limit=bedrock_env.bedrock_rpm_limit,
)

In [ ]:
logger.info(f" Resonse from bedrock agent: {final_response.response_body}")